In [107]:
import pandas as pd
import numpy as np
import os
import re
# from icecream import ic
from collections import defaultdict
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
# import pickle

In [108]:
# embeds_file = '/home/varunaja/mri_pet/ready_data/ADNI_ABTAU_MOREFEATURES.pkl'
# with open(embeds_file, 'rb') as file:
#     # Load the data from the pickle file
#     data = pickle.load(file)

In [109]:
def code_data_type(df, meta, percent_missing):
    for i, col in enumerate(df.columns):
        if col == 'ID':
            continue
        # print(col)
        # print(meta1[meta1['name'] == col]['type'].iloc[0])
        meta[col] = {}
        meta[col]['name'] = col
        meta[col]['unique_values'] = list(df[col].dropna().unique())
        meta[col]['num_unique_values'] = len(meta[col]['unique_values'])
        meta[col]['length'] = 1
        mydtype = df[col].dtype
        if 'category' in str(mydtype):
            meta[col]['type'] = 'C'
        else:
            meta[col]['type'] = 'N'
        if col == 'img_MRI_T1':
            meta[col]['type'] = 'M'
        meta[col]['missing_rate'] = round(percent_missing[col], 2)
    df = pd.DataFrame(meta).T[['name', 'missing_rate', 'type', 'num_unique_values', 'length', 'unique_values']]
    return df

# Load Data

In [110]:
# data = pd.read_csv('/home/varunaja/mri_pet/adrd_tool_varuna/adrd_transformer/data/adni_data_single.csv')
# data = pd.read_csv('/home/varunaja/mri_pet/adrd_tool_varuna/adrd_transformer/data/oasis_data_single.csv')
data = pd.read_csv('../../../adrd_tool/data_varuna/data/train_val_0504.csv')
data.head()

,FS_MTL_VOLUME,FS_TEMPORAL_VOLUME,FS_PARIETAL_VOLUME,FS_OCCIPITAL_VOLUME,FS_FRONTAL_VOLUME,FS_ctx_lh_entorhinal_volume,FS_ctx_rh_entorhinal_volume,FS_left_hippocampus_volume,FS_right_hippocampus_volume,FS_left_amygdala_volume,FS_right_amygdala_volume,FS_ctx_lh_parahippocampal_volume,FS_ctx_rh_parahippocampal_volume,FS_ctx_lh_inferiortemporal_volume,FS_ctx_rh_inferiortemporal_volume,FS_ctx_lh_fusiform_volume,FS_ctx_rh_fusiform_volume,FS_ctx_lh_middletemporal_volume,FS_ctx_rh_middletemporal_volume,FS_ctx_lh_superiortemporal_volume,FS_ctx_rh_superiortemporal_volume,FS_ctx_lh_transversetemporal_volume,FS_ctx_rh_transversetemporal_volume,FS_ctx_lh_isthmuscingulate_volume,FS_ctx_rh_isthmuscingulate_volume,FS_ctx_lh_inferiorparietal_volume,FS_ctx_rh_inferiorparietal_volume,FS_ctx_lh_precuneus_volume,FS_ctx_rh_precuneus_volume,FS_ctx_lh_superiorparietal_volume,FS_ctx_rh_superiorparietal_volume,FS_ctx_lh_supramarginal_volume,FS_ctx_rh_supramarginal_volume,FS_ctx_lh_posteriorcingulate_volume,FS_ctx_rh_posteriorcingulate_volume,FS_ctx_lh_postcentral_volume,FS_ctx_rh_postcentral_volume,FS_ctx_lh_lateraloccipital_volume,FS_ctx_rh_lateraloccipital_volume,FS_ctx_lh_lingual_volume,FS_ctx_rh_lingual_volume,FS_ctx_lh_pericalcarine_volume,FS_ctx_rh_pericalcarine_volume,FS_ctx_lh_cuneus_volume,FS_ctx_rh_cuneus_volume,FS_ctx_lh_caudalanteriorcingulate_volume,FS_ctx_rh_caudalanteriorcingulate_volume,FS_ctx_lh_caudalmiddlefrontal_volume,FS_ctx_rh_caudalmiddlefrontal_volume,FS_ctx_lh_lateralorbitofrontal_volume,FS_ctx_rh_lateralorbitofrontal_volume,FS_ctx_lh_medialorbitofrontal_volume,FS_ctx_rh_medialorbitofrontal_volume,FS_ctx_lh_parsopercularis_volume,FS_ctx_rh_parsopercularis_volume,FS_ctx_lh_parsorbitalis_volume,FS_ctx_rh_parsorbitalis_volume,FS_ctx_lh_parstriangularis_volume,FS_ctx_rh_parstriangularis_volume,FS_ctx_lh_precentral_volume,FS_ctx_rh_precentral_volume,FS_ctx_lh_rostralanteriorcingulate_volume,FS_ctx_rh_rostralanteriorcingulate_volume,FS_ctx_lh_rostralmiddlefrontal_volume,FS_ctx_rh_rostralmiddlefrontal_volume,FS_ctx_lh_superiorfrontal_volume,FS_ctx_rh_superiorfrontal_volume,FS_ctx_lh_insula_volume,FS_ctx_rh_insula_volume,FS_wm_hypointensities_volume,blood_PLASMAPTAU181,blood_t-TAU,blood_AB42_AB40,ID,his_BIRTHMO,his_BIRTHYR,his_SEX,his_HISPANIC,his_PRIMLANG,his_EDUC,his_MARISTAT,his_LIVSIT,his_INDEPEND,his_RESIDENC,his_HANDED,his_NACCAGE,his_NACCNIHR,apoe_NACCNE4S,his_NACCFAM,his_NACCMOM,his_NACCDAD,med_ANYMEDS,med_NACCAMD,med_NACCAHTN,med_NACCHTNC,med_NACCACEI,med_NACCAAAS,med_NACCBETA,med_NACCCCBS,med_NACCDIUR,med_NACCVASD,med_NACCANGI,med_NACCLIPL,med_NACCNSD,med_NACCAC,med_NACCADEP,med_NACCAPSY,med_NACCAANX,med_NACCADMD,med_NACCPDMD,med_NACCEMD,med_NACCEPMD,med_NACCDBMD,his_TOBAC30,his_TOBAC100,his_SMOKYRS,his_PACKSPER,his_QUITSMOK,his_ALCOCCAS,his_ALCFREQ,his_CVHATT,his_CVAFIB,his_CVANGIO,his_CVBYPASS,his_CVPACDEF,his_CVCHF,his_CVANGINA,his_CVHVALVE,his_CBSTROKE,his_CBTIA,his_PD,his_SEIZURES,his_TBI,his_TBIBRIEF,his_TBIEXTEN,his_TBIWOLOS,his_TBIYEAR,his_DIABETES,his_DIABTYPE,his_HYPERTEN,his_HYPERCHO,his_B12DEF,his_THYROID,his_ARTHRIT,his_ARTHTYPE,his_ARTHUPEX,his_ARTHLOEX,his_ARTHSPIN,his_ARTHUNK,his_INCONTU,his_INCONTF,his_APNEA,his_RBD,his_INSOMN,his_ALCOHOL,his_PTSD,his_BIPOLAR,his_SCHIZ,his_DEP2YRS,his_ANXIETY,his_OCD,his_NPSYDEV,his_PSYCDIS,his_NACCTBI,ph_HEIGHT,ph_WEIGHT,ph_NACCBMI,ph_BPSYS,ph_BPDIAS,ph_HRATE,ph_VISION,ph_VISCORR,ph_VISWCORR,ph_HEARING,ph_HEARAID,ph_HEARWAID,npiq_DEL,npiq_HALL,npiq_AGIT,npiq_DEPD,npiq_ANX,npiq_ELAT,npiq_APA,npiq_DISN,npiq_IRR,npiq_MOT,npiq_NITE,npiq_APP,gds_SATIS,gds_DROPACT,gds_EMPTY,gds_BORED,gds_SPIRITS,gds_AFRAID,gds_HAPPY,gds_HELPLESS,gds_STAYHOME,gds_MEMPROB,gds_WONDRFUL,gds_WRTHLESS,gds_ENERGY,gds_HOPELESS,gds_BETTER,gds_NACCGDS,faq_BILLS,faq_TAXES,faq_SHOPPING,faq_GAMES,faq_STOVE,faq_MEALPREP,faq_EVENTS,faq_PAYATTN,faq_REMDATES,faq_TRAVEL,exam_PARKSIGN,exam_RESTTRL,exam_RESTTRR,exam_SLOWINGL,exam_SLOWINGR,exam_RIGIDL,exam_RIGIDR,exam_BRADY,exam_PARKGAIT,exam_POSTINST,exam_CVDSIGNS,e

In [111]:
data.drop(columns=['his_BIRTHMO', 'his_BIRTHYR'], inplace = True)

In [112]:
# data.drop(columns = ['cd_DIAGNOSIS'], inplace = True)

In [113]:
# demog = data.columns[data.columns.str.contains('his')].to_list()
demog_his = data.columns[data.columns.str.contains('his')].to_list()
# diagnosis = ['cd_DIAGNOSIS']
mmse = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MMSE')].to_list()
cdr = data.columns[data.columns.str.contains('cdr')].to_list()
cvd = data.columns[data.columns.str.contains('cvd')].to_list()
physical = data.columns[data.columns.str.contains('ph')].to_list()
moca = data.columns[data.columns.str.contains('bat') & data.columns.str.contains('MOCA')].to_list()
np = data.columns[(data.columns.str.contains('bat')) & (~data.columns.str.contains('MOCA')) & (~data.columns.str.contains('MMSE'))].to_list()
np_cat = ['bat_UDSBENRS', 'bat_COGSTAT', 'bat_CRAFTCUE']
np_num = [item for item in np if item not in np_cat]
gds = data.columns[data.columns.str.contains('gds')].to_list()
npiq = data.columns[data.columns.str.contains('npiq')].to_list()
faq = data.columns[data.columns.str.contains('faq')].to_list()
exam = data.columns[data.columns.str.contains('exam')].to_list()
meds = data.columns[data.columns.str.contains('med')].to_list()
dx = data.columns[data.columns.str.contains('cd_NACCUDSD')].to_list()
apoe = data.columns[data.columns.str.contains('apoe')].to_list()
csf = data.columns[data.columns.str.contains('csf')].to_list()
blood = data.columns[data.columns.str.contains('blood')].to_list()
# hx = data.columns[data.columns.str.contains('iha')].to_list()
# labels = ['amy_label', 'tau_label', 'NC', 'MCI', 'DE']
labels = ['amy_label', 'tau_label']
volumes = data.columns[data.columns.str.contains('FS')].to_list()

In [114]:
data.dropna(subset = ['bat_MOCATOTS'])[['bat_NACCMOCA', 'bat_MOCATOTS']]

,bat_NACCMOCA,bat_MOCATOTS
261,25.0,25.0
262,18.0,18.0
263,25.0,25.0
265,28.0,28.0
266,25.0,25.0
...,...,...
4759,22.0,22.0
4760,24.0,24.0
4761,18.0,18.0
4767,20.0,20.0


# All training data

In [115]:
all_columns = demog_his + mmse + volumes + cdr + cvd + physical + moca + np + gds + npiq + faq + exam + meds + dx + apoe + csf + blood + labels

numerical = ['his_EDUC'] + mmse + physical + volumes + cdr + blood + np_num + ['his_NACCAGE', 'med_NACCAMD', 'his_SMOKYRS', 
                                                                   'his_PACKSPER', 'his_QUITSMOK', 'his_ALCFREQ', 
                                                                   'his_TBIYEAR', 'exam_GAITNPH',
                                                                   'bat_MOCANAMI', 'bat_MOCASER7', 'bat_MOCAREGI',
                                                                   'bat_COGSTAT', 'bat_MOCARECN', 'bat_MOCARECR',
                                                                   'bat_MOCATOTS',
                                                                   'bat_MOCARECC', 'bat_NACCMOCA', 
                                                                   'bat_UDSBENRS', 'bat_TRAILA', 
                                                                   'bat_TRAILB', 'gds_NACCGDS'] 

# exclude from np 

# imaging = ['img_MRI_T1']

for col in all_columns:
    if col not in numerical: # and col not in imaging:
        data[col] = data[col].astype('category')

for col in numerical:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [116]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
filtered_percent_missing = {key: value for key, value in percent_missing.items() if value > 0.89}

filtered_percent_missing

{'blood_PLASMAPTAU181': 0.9956900590274524,
 'blood_t-TAU': 0.9755457696992411,
 'blood_AB42_AB40': 0.9757331584371779,
 'his_QUITSMOK': 0.9195165370561229,
 'his_TBIYEAR': 0.9719853836784409,
 'his_DIABTYPE': 0.9747025203785252,
 'his_ARTHTYPE': 0.9191417595802492,
 'ph_HEARWAID': 0.9298229176426497,
 'bat_PENTAGON': 0.9797620163028202,
 'bat_MEMTIME': 0.9809800430994097,
 'bat_DIGIF': 0.9802304881476623,
 'bat_DIGIFLEN': 0.9560573409538087,
 'bat_DIGIB': 0.9802304881476623,
 'bat_DIGIBLEN': 0.9560573409538087,
 'bat_BOSTON': 0.9578375339642088,
 'csf_AMYLCSF': 0.89946594209688,
 'csf_CSFTAU': 0.9022767731659327,
 'FDGAD': 0.8927199475311534,
 'SCANDATE.FDG': 0.8927199475311534,
 'NACCADC': 0.9642087510540617,
 'MRIT1': 0.9642087510540617,
 'MRIT2': 0.9642087510540617,
 'MRIFLAIR': 0.9642087510540617,
 'MRIFIELD': 0.9642087510540617,
 'MRIMANU': 0.9642087510540617,
 'MRIMODL': 0.9642087510540617,
 'NACCNMRI': 0.9642087510540617,
 'NACCAUTP': 0.9410662419188607,
 'SCANDATE.MRI': 0.9642

In [117]:
# filtered_percent_missing = {key: value for key, value in filtered_percent_missing.items() if not key.startswith('blood')}
# filtered_percent_missing

In [118]:
to_drop = [ 'bat_MOCATOTS',
            'FDGAD',
 'SCANDATE.FDG',
 'NACCADC',
 'MRIT1',
 'MRIT2',
 'MRIFLAIR',
 'MRIFIELD',
 'MRIMANU',
 'MRIMODL',
 'NACCNMRI',
 'NACCAUTP',
 'SCANDATE.MRI',
 'NACCMRFI']

In [119]:
# df = data.drop(columns=filtered_percent_missing.keys())
data.drop(columns=to_drop, inplace = True)

In [120]:
for i, col in enumerate(data.columns):
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        # print(col)
        print(data[col].value_counts())

female    5939
male      4734
Name: his_SEX, dtype: int64
no     9534
yes     556
Name: his_HISPANIC, dtype: int64
0.0    8767
1.0     284
5.0     101
6.0      67
2.0       8
3.0       4
4.0       2
Name: his_PRIMLANG, dtype: int64
0.0    7212
2.0    1284
1.0     975
4.0     496
5.0      91
3.0      50
Name: his_MARISTAT, dtype: int64
1.0    3718
0.0    1038
3.0     349
2.0     306
4.0      47
Name: his_LIVSIT, dtype: int64
0.0    3817
1.0     999
2.0     412
3.0     172
Name: his_INDEPEND, dtype: int64
0.0    9606
1.0     184
2.0      61
3.0      52
Name: his_RESIDENC, dtype: int64
1.0    5024
0.0     530
2.0     107
Name: his_HANDED, dtype: int64
whi    9044
blk     658
asi     266
mul     136
ind      37
haw       5
Name: his_NACCNIHR, dtype: int64
0.0    2443
1.0    1540
2.0     336
Name: apoe_NACCNE4S, dtype: int64
1.0    2479
0.0    1497
Name: his_NACCFAM, dtype: int64
0.0    5465
1.0    4343
Name: his_NACCMOM, dtype: int64
0.0    7540
1.0    2215
Name: his_NACCDAD, dtype: int64


In [121]:
list(data.columns)

['FS_MTL_VOLUME',
 'FS_TEMPORAL_VOLUME',
 'FS_PARIETAL_VOLUME',
 'FS_OCCIPITAL_VOLUME',
 'FS_FRONTAL_VOLUME',
 'FS_ctx_lh_entorhinal_volume',
 'FS_ctx_rh_entorhinal_volume',
 'FS_left_hippocampus_volume',
 'FS_right_hippocampus_volume',
 'FS_left_amygdala_volume',
 'FS_right_amygdala_volume',
 'FS_ctx_lh_parahippocampal_volume',
 'FS_ctx_rh_parahippocampal_volume',
 'FS_ctx_lh_inferiortemporal_volume',
 'FS_ctx_rh_inferiortemporal_volume',
 'FS_ctx_lh_fusiform_volume',
 'FS_ctx_rh_fusiform_volume',
 'FS_ctx_lh_middletemporal_volume',
 'FS_ctx_rh_middletemporal_volume',
 'FS_ctx_lh_superiortemporal_volume',
 'FS_ctx_rh_superiortemporal_volume',
 'FS_ctx_lh_transversetemporal_volume',
 'FS_ctx_rh_transversetemporal_volume',
 'FS_ctx_lh_isthmuscingulate_volume',
 'FS_ctx_rh_isthmuscingulate_volume',
 'FS_ctx_lh_inferiorparietal_volume',
 'FS_ctx_rh_inferiorparietal_volume',
 'FS_ctx_lh_precuneus_volume',
 'FS_ctx_rh_precuneus_volume',
 'FS_ctx_lh_superiorparietal_volume',
 'FS_ctx_rh_supe

In [122]:
df = code_data_type(data, meta, percent_missing)

In [123]:
# df.to_csv('../meta_files/train_meta_file_imaging_0422.csv', index=False)
df.to_csv('../meta_files/train_meta_file_0504.csv', index=False)